# Multi-Agent Collaboration System

This notebook demonstrates how to build a multi-agent collaborative system using LlamaIndex, where different specialized agents work together to complete complex tasks.

## Prerequisites
- An OpenAI API key
- Notion integration token (optional, for Notion research)
- Understanding of LlamaIndex workflows and agents

# Warning
This notebook first run can take up to 5 minutes to run due to the Notion API. After the first run, it will be much faster as the data will be cached. If you want to speed up the process, you can use a local file or a different data source instead of Notion.

## Setup and Installation

First, let's install the required packages. Run the cell below to install all dependencies needed for this notebook.

In [ ]:
# Install required packages
!pip install llama-index llama-index-llms-openai llama-index-readers-notion python-dotenv tavily-python nest-asyncio openai

# Verify installations
import importlib

def check_package(package_name):
    try:
        importlib.import_module(package_name)
        return True
    except ImportError:
        return False

packages = {
    "llama_index": "llama-index core",
    "llama_index.llms.openai": "OpenAI integration",
    "llama_index.readers.notion": "Notion reader",
    "tavily": "Tavily search",
    "nest_asyncio": "Nested async loop support",
    "openai": "OpenAI API"
}

all_installed = True
for package, display_name in packages.items():
    installed = check_package(package)
    print(f"{display_name}: {'✅ Installed' if installed else '❌ Not installed'}")
    all_installed = all_installed and installed

if all_installed:
    print("\n✅ All required packages are installed!")
else:
    print("\n⚠️ Some packages are missing. Run the installation command again.")

## Environment Setup

Load environment variables from the `.env` file.

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get API keys from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
NOTION_INTEGRATION_TOKEN = os.getenv("NOTION_INTEGRATION_TOKEN")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# Set environment variables for compatibility with libraries that expect them
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY or ""
os.environ["NOTION_INTEGRATION_TOKEN"] = NOTION_INTEGRATION_TOKEN or ""
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY or ""

# Verify API keys are set
if not OPENAI_API_KEY:
    print("⚠️ Warning: OPENAI_API_KEY is not set in .env file")
else:
    print("✅ OpenAI API key is set")

if not NOTION_INTEGRATION_TOKEN:
    print("⚠️ Warning: NOTION_INTEGRATION_TOKEN is not set in .env file (optional for Notion research)")
else:
    print("✅ Notion integration token is set")

if not TAVILY_API_KEY:
    print("⚠️ Warning: TAVILY_API_KEY is not set in .env file (optional for web search)")
else:
    print("✅ Tavily API key is set")

## Setup Language Model

We'll configure OpenAI's language model for our agents.

In [ ]:
from llama_index.llms.openai import OpenAI

# Configure OpenAI model with API key from environment
llm = OpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
print("✅ Language model configured")

## Define Research Tools

Let's define tools our agents can use to gather information.

In [ ]:
from tavily import AsyncTavilyClient 
import pickle
import time
import os

from llama_index.readers.notion import NotionPageReader

async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=TAVILY_API_KEY)
    return str(await client.search(query))


async def notion_search(query: str) -> str:
    """Search in Notion with caching for both documents and vector index."""
    # Specify your Notion page IDs here
    page_ids = ["9917363395904835a604ca7a6a358579"]  # Replace with your actual page ID
    cache_folder = "cache"
    os.makedirs(cache_folder, exist_ok=True)
    
    # Cache files
    doc_cache_file = os.path.join(cache_folder, "notion_data.pkl")
    index_cache_file = os.path.join(cache_folder, "vector_index.pkl")
    
    one_week_seconds = 7 * 24 * 60 * 60
    update_cache = True

    # Check documents cache
    if os.path.exists(doc_cache_file):
        file_age = time.time() - os.path.getmtime(doc_cache_file)
        if file_age < one_week_seconds:
            update_cache = False

    if update_cache:
        print("Fetching new data from Notion...", flush=True)
        documents = NotionPageReader(integration_token=NOTION_INTEGRATION_TOKEN).load_data(
            page_ids=page_ids
        )
        with open(doc_cache_file, "wb") as f:
            pickle.dump(documents, f)
        # If documents were updated, we also recalc the index.
        update_cache_index = True
    else:
        print("Loading data from cache...", flush=True)
        with open(doc_cache_file, "rb") as f:
            documents = pickle.load(f)
        update_cache_index = not os.path.exists(index_cache_file)  # Recalc index if not cached

    # Setup text splitter (splitting the text into chunks)
    from llama_index.core.node_parser import SentenceSplitter
    from llama_index.core import Settings
    text_splitter = SentenceSplitter(chunk_size=100, chunk_overlap=10)
    Settings.text_splitter = text_splitter

    # Load or build vector index
    if update_cache_index:
        from llama_index.core import VectorStoreIndex
        index = VectorStoreIndex.from_documents(documents, transformations=[text_splitter])
        with open(index_cache_file, "wb") as f:
            pickle.dump(index, f)
    else:
        print("Loading vector index from cache...", flush=True)
        with open(index_cache_file, "rb") as f:
            index = pickle.load(f)
    
    # Limit results to the top 3 matches
    query_engine = index.as_query_engine(llm=llm, similarity_top_k=3)
    response = query_engine.query(query)
     
    return str(response)

## Create Multi-Agent System

Now we'll define multiple specialized agents that will collaborate on tasks.

In [ ]:
from llama_index.core.agent import FunctionCallingAgent as GenericFunctionCallingAgent
from llama_index.core.tools import FunctionTool

# Convert our search functions into tools
search_web_tool = FunctionTool.from_defaults(fn=search_web)
search_notion_tool = FunctionTool.from_defaults(fn=notion_search)

# Research agent - responsible for information gathering
research_agent = GenericFunctionCallingAgent.from_tools(
    tools=[search_web_tool, search_notion_tool],
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
    system_prompt="You are an agent that researches information from various sources including Notion and the web. "\
               "Your goal is to gather comprehensive and accurate information on a given topic."
)

# Writing agent - responsible for composing responses
write_agent = GenericFunctionCallingAgent.from_tools(
    tools=[],
    llm=llm,
    verbose=False,
    allow_parallel_tool_calls=False,
    system_prompt="You are an agent that writes clear, detailed, and well-structured answers "\
               "based on research provided to you. Format your responses appropriately."
)

# Review agent - responsible for quality assurance
review_agent = GenericFunctionCallingAgent.from_tools(
    tools=[],
    llm=llm,
    verbose=False,
    allow_parallel_tool_calls=False,
    system_prompt="You are an agent that critically reviews answers for accuracy, clarity, and completeness. "\
               "You should identify any issues, inaccuracies, or areas for improvement."
)

## Define Workflow Events and Steps

We'll now set up the workflow that our agents will follow to collaborate.

In [ ]:
from llama_index.core.workflow import Event, Context
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
)

class ResearchEvent(Event):
    prompt: str

class ReviewEvent(Event):
    answer: str

class ReviewResults(Event):
    review: str

class RewriteEvent(Event):
    review: str

class WriteEvent(Event):
    pass

class MultiAgentFlowWithReflection(Workflow):

    @step
    async def setup(self, ctx: Context, ev: StartEvent) -> ResearchEvent:
        self.research_agent = ev.research_agent
        self.write_agent = ev.write_agent
        self.review_agent = ev.review_agent
        # Initialize rewriting counter and store the original prompt
        await ctx.set("rewrite_count", 0)
        await ctx.set("prompt", ev.prompt)
        return ResearchEvent(prompt=ev.prompt)

    @step
    async def research(self, ctx: Context, ev: ResearchEvent) -> WriteEvent:
        await ctx.set("prompt", ev.prompt)
        result = self.research_agent.chat(
            f"Gather some information that another agent will use to write an answer about this topic: <topic>{ev.prompt}</topic>. "
            "Just include the facts without making it into a full answer."
        )
        print("Research result:", result)
        await ctx.set("research", str(result))
        return WriteEvent()

    @step
    async def write(self, ctx: Context, ev: WriteEvent | RewriteEvent) -> ReviewEvent:
        original_prompt = await ctx.get("prompt")
        research_info = await ctx.get("research")
        prompt = (
            f"Write a detailed, clear, and direct answer addressing the question: "
            f"<question>{original_prompt}</question>. Use the following research as supporting information: "
            f"<research>{research_info}</research>"
        )
        if isinstance(ev, RewriteEvent):
            print("Doing a rewrite!")
            prompt += (
                f" Note: This answer has been reviewed and the reviewer provided the following feedback that "
                f"should be taken into account: <review>{ev.review}</review>"
            )
        result = self.write_agent.chat(prompt)
        return ReviewEvent(answer=str(result))

    @step
    async def review(self, ctx: Context, ev: ReviewEvent) -> StopEvent | RewriteEvent:
        result = self.review_agent.chat(f"Review this answer: {ev.answer}")
        original_prompt = await ctx.get("prompt")
        research_info = await ctx.get("research")
        rewrite_count = await ctx.get("rewrite_count")
        rewrite_count += 1
        await ctx.set("rewrite_count", rewrite_count)
        try_again = llm.complete(
            f"This is a review of an answer written by an agent. If you think this review is bad enough "
            f"that the agent should try again, respond with just the word RETRY. If the review is good, reply "
            f"with just the word CONTINUE. Here's the review: <review>{str(result)}</review>"
        )
        if try_again.text == "RETRY":
            if rewrite_count > 3:
                print("Maximum rewrite attempts reached. Finishing the flow.")
                return StopEvent(result=ev.answer)
            print("Reviewer said try again")
            return RewriteEvent(
                review=(
                    f"{str(result)}\n"
                    f"Original prompt: {original_prompt}\n"
                    f"Research info: {research_info}"
                )
            )
        else:
            print("Reviewer thought it was good!")
            return StopEvent(result=ev.answer)

## Run the Multi-Agent Workflow

Let's run our collaborative multi-agent system on a sample question.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

workflow = MultiAgentFlowWithReflection(timeout=1200, verbose=True)
handler = workflow.run(
    prompt="The roles of trees on Earth?",  # You can change this prompt to any question you want to ask
    research_agent=research_agent,
    write_agent=write_agent,
    review_agent=review_agent
)

final_result = await handler
print("\n==== Final Answer ====")
print(final_result)

## Customizing the Multi-Agent System

You can modify the system by changing the prompts, adding new agents, or adjusting the workflow.

In [ ]:
# Example: Run the workflow with a different prompt
another_workflow = MultiAgentFlowWithReflection(timeout=120, verbose=True)
another_handler = another_workflow.run(
    prompt="What are the key principles of sustainable architecture?",  # Different question
    research_agent=research_agent,
    write_agent=write_agent,
    review_agent=review_agent
)

another_result = await another_handler
print("\n==== Final Answer ====")
print(another_result)

## Conclusion

In this notebook, we've demonstrated how to build a multi-agent collaborative system using LlamaIndex that:

1. Uses specialized agents for different tasks (research, writing, and review)
2. Implements a workflow for agents to collaborate sequentially
3. Includes a feedback loop where content is reviewed and potentially rewritten
4. Leverages external knowledge sources like Notion and web search

This approach can be extended to more complex workflows and specialized agents for various domains and use cases.

## Troubleshooting Tips

If you encounter issues:

1. **API Keys**: Verify that your API keys are correct and have the necessary permissions
2. **Network Issues**: Make sure you have internet connectivity for web searches
3. **Notion Integration**: For Notion integration, ensure your token has access to the specified pages
4. **Async Runtime**: If you encounter async-related errors, ensure nest_asyncio is properly applied
5. **Dependencies**: Make sure all required packages are installed:
   ```
   pip install llama-index llama-index-llms-openai llama-index-readers-notion python-dotenv tavily-python nest-asyncio openai
   ```